<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Método entropía cruzada</center></span>

<center>Implementación</center>

<figure>
<center>
<img src="../Imagenes/trainer.png" width="800" height="800" align="center"/>
</center>
</figure>


Fuente: Alvaro Montenegro

##   <span style="color:blue">Profesores</span>

### Coordinador

- Campo Elías Pardo, PhD, cepardot@unal.edu.co

### Conferencistas

- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Referencias</span>

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [Turing Paper 1936](http://www.thocp.net/biographies/papers/turing_oncomputablenumbers_1936.pdf)
1. [Solving a Reinforcement Learning Problem Using Cross-Entropy Method](https://towardsdatascience.com/solving-a-reinforcement-learning-problem-using-cross-entropy-method-23d9726a737)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Implementación básica del Método Entropia Cruzada](#Implementación-básica-del-Método-Entropia-Cruzada)


## <span style="color:blue">Introducción</span>

La entropía cruzada se considera un algoritmo evolutivo: algunos individuos se muestrean de una población, y solo los de `élite` gobiernan las características de las generaciones futuras.

Esencialmente, lo que hace el método de `entropía-cruzada`(cross-entropy) es tomar un montón de entradas, ver las salidas producidas, elegir las entradas que han llevado a las mejores salidas y ajustar el agente hasta que estemos satisfechos con las salidas que vemos.

Antes de describir el método técnicamente vamos a repasar los conceptos básicos del aprendizaje reforzado.

<figure>
<center>
<img src="../Imagenes/agente-ambiente.png" width="600" height="500" align="center"/>
</center> 
</figure>


Fuente: Alvaro Montenegro

- Un conjunto de `acciones` que se permite sean ejecutadas en el ambiente.
- El tamaño y bordes de las `observaciones` que el ambiente le provee al agente.
- Un método *step* para ejecutar un acción. El método regresa la nueva observación, la `recompensa` y la indicación de si el `episodio` ha terminado (*done*).
- Un método *reset* que retorna al ambiente a su *estado inicial* y entrega la primera observación.

## <span style="color:blue">Implementación básica del Método Entropia Cruzada</span>

Primero demos una mirada al espacio de observaciones del ambiente. El método render de un objeto de tipo Env renderiza el espacio de acciones. Para interpretrar *S* es start, *F* es free, *H*  es hole y *G* es goal.

### importa librerías

In [31]:
import gym, gym.spaces
from collections import namedtuple
import numpy as np

#from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim

import gym

### Envuelve el espacio de acciones para el ambiente FrozenLake

Con esta clase se envuelve el epacio de acciones para convertirlo en un espacio de tipo de tal manera que sea compatible con el tipo de espacio de CartPole, que vamos a estudiar en la siguiente lección. El nuevo tipo de observation será de  *Box* y contendrá un vector de tamaño 16, de tipo *onehot*. 

In [32]:
import gym

class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape=shape, dtype=np.float32)
        
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res
    

### Clase Net 

In [33]:
import torch
from torch import nn

class Net_basic(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net_basic, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
        self.optimizer = self.configure_optimizers()
        self.loss = self.configure_looses()

    def forward(self, x):
        return self.net(x)
 
    def configure_optimizers(self):
        raise NotImplementedError
    
    def configure_looses(self):
        raise NotImplementedError
    
    def training_step(self, train_batch):
        raise NotImplementedError

In [34]:
import torch

class Net(Net_basic): 
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__(obs_size, hidden_size, n_actions)
           
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def configure_looses(self):
        loss = torch.nn.CrossEntropyLoss()
        return loss
    
    def training_step(self, train_batch):
        x, y = train_batch
        y_hat = self.net(x)
        loss = self.loss(y_hat, y)
        #self.log('train_loss', loss)
        return loss

In [35]:
import numpy as np

class Trainer(object):
    def __init__(self, model, writer, verbose=True):
        self.model = model
        self.writer = writer
        self.verbose = verbose
        
    def fit(self, iter_no, train_dataloader, reward_bound, reward_mean):
        loss_l = []
        for batch in train_dataloader:
            self.model.optimizer.zero_grad()
            loss = self.model.training_step(batch)
            loss_l.append(loss.item())
            loss.backward()
            self.model.optimizer.step()
            
        mean_loss = np.mean(loss_l)
        
        # escribe en el log de writer
        self.writer.add_scalar("perdida", mean_loss, iter_no)
        self.writer.add_scalar("recompensa_promedio", reward_mean, iter_no)
        self.writer.add_scalar("recompensa_frontera", reward_bound, iter_no)
        
        # escribe en la pantalla
        if self.verbose:
            print("%d: pérdida promedio =%.3f, recompensa promedio=%.1f, cota recompensa=%.1f" % (
            iter_no, mean_loss, reward_mean, reward_bound))
            


### Clase iterable Batch

In [36]:
from collections import namedtuple
import numpy as np

#  tupla para retornar todos los datos de un episodio completo
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
# tupla para para almacenar las parejas (observación, acción) de cada paso 
# en un episodio
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


class Batch(object):
    '''
    Implementa la generación iterativa de lotes de  datos
    '''
    def __init__(self, env, net, batch_size):
        self.env = env
        self.net = net
        self.batch_size = batch_size
   
    # hace la clase iterable       
    def __iter__(self):
        return self
    
    # define iterador
    def __next__(self):
        # lista que contendrá el lote  de datos a entregar
        batch = [] 
        # recompensa de cada episodio
        episode_reward = 0.0 
        # lista de parejas (observación, acción) de cada episodio
        episode_steps = [] 
        
        # reinicia el ambiente, para empezar a generar datos
        # recibe la primera observación
        obs = self.env.reset()
        # alias para la función softmax
        sm = nn.Softmax(dim=1)
        
        # ciclo para generar lote(batch) de datos
        while True:
            # convierte obs a un tensor. debe pasar como lista
            obs_v = torch.FloatTensor(np.array([obs]))
            # calcula el tensor de puntaje para las acciones: self.net(obs_v)
            # Transforma los puntajes entregado por la red en una distribución
            # de probabilidad, la cual viene en un tensor
            act_probs_v = sm(self.net(obs_v))
            # extrae la distribución del tensor a d-array de  Numpy
            act_probs = act_probs_v.data.numpy()[0]
            # selecciona una acción aleatoriamente usando la distribución
            action = np.random.choice(len(act_probs), p=act_probs)
            # entrega la acción al ambiente y recibe respuesta del ambiete
            next_obs, reward, is_done, _ = self.env.step(action)
            # actualzia la recompensa
            episode_reward += reward
            # agrega la pareja (observación, acción) a la lista de pasos
            episode_steps.append(EpisodeStep(observation=obs, action=action))
            # al terminar el episodio
            if is_done:
                # agrega los datos al batch: (recompensa, lista de parejas (obs, acción))
                batch.append(Episode(reward=episode_reward, steps=episode_steps))
                # reinicia objetos para el siguiente episodio
                episode_reward = 0.0
                episode_steps = []
                next_obs = env.reset()
                # si completo el lote de datos, lo retorna y termina
                if len(batch) == self.batch_size:
                    return batch
            obs = next_obs

#### Prueba del iterador de lotes

In [7]:
# selecciona un ambiente FrozenLake
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes
BATCH_SIZE = 2
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# extrae el primer lote de datos
dato = next(batch)
dato

[Episode(reward=0.0, steps=[EpisodeStep(observation=array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=2), EpisodeStep(observation=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=1), EpisodeStep(observation=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=3), EpisodeStep(observation=array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=0), EpisodeStep(observation=array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=0), EpisodeStep(observation=array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=0), EpisodeStep(observation=array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32), action=0), EpisodeStep(observation=array([0., 0., 0., 0., 1., 0., 0., 0.,

In [8]:
# selecciona un ambiente CartPole
env = gym.make("CartPole-v1")

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes
BATCH_SIZE = 5
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# extrae el primer lote de datos
dato = next(batch)
dato

[Episode(reward=15.0, steps=[EpisodeStep(observation=array([-0.04493422,  0.04909142, -0.00595491, -0.02855235], dtype=float32), action=0), EpisodeStep(observation=array([-0.04395239, -0.14594463, -0.00652596,  0.2622458 ], dtype=float32), action=0), EpisodeStep(observation=array([-0.04687128, -0.3409728 , -0.00128104,  0.55286324], dtype=float32), action=0), EpisodeStep(observation=array([-0.05369074, -0.5360768 ,  0.00977622,  0.8451423 ], dtype=float32), action=1), EpisodeStep(observation=array([-0.06441227, -0.34108955,  0.02667907,  0.5555496 ], dtype=float32), action=0), EpisodeStep(observation=array([-0.07123406, -0.53657573,  0.03779006,  0.85651726], dtype=float32), action=1), EpisodeStep(observation=array([-0.08196558, -0.3419885 ,  0.05492041,  0.5759525 ], dtype=float32), action=0), EpisodeStep(observation=array([-0.08880535, -0.5378356 ,  0.06643945,  0.8854186 ], dtype=float32), action=1), EpisodeStep(observation=array([-0.09956206, -0.3436755 ,  0.08414783,  0.6143399 ],

## Clase Agent

In [37]:
import numpy as np

from torch.utils.data import Dataset, TensorDataset, DataLoader

class Agent:
    def __init__(self, batch_iterator, percentile, batch_size=1):
        self.batch_iterator =  batch_iterator
        self.percentile = percentile
        self.writer = writer # para escribir en Tensorboard
        self.batch_size = batch_size # batch size para los dataloaders
        
    def take_action(self): 
        # toma un lote de datos del iterador de lotes
        batch = next(self.batch_iterator)
        # extrae todas las recompensas del batch y hace una lista con ellas
        rewards = list(map(lambda s: s.reward, batch))
        # calcula la cota inferior para extraer los episodios élite (por defecto percentil 70)
        reward_bound = np.percentile(rewards, self.percentile)
        # calcula la recompensa promedio del lote de datos
        reward_mean = float(np.mean(rewards))
        
        # extrae las observaciones y las respectivas acciones de los episodios élite
        train_obs = []
        train_act = []
        for example in batch:
            if example.reward < reward_bound:
                continue
            # de cada episodio élite extrae todas las parejas (observación, acción)
            # agregando las observaciones en la lista de observaciones
            train_obs.extend(map(lambda step: step.observation, example.steps))
            # y las acciones en la lista de acciones
            train_act.extend(map(lambda step: step.action, example.steps))
            
        # convierte listas a tensores
        train_obs_v = torch.FloatTensor(train_obs)
        train_act_v = torch.LongTensor(train_act)
        # crea el dataset
        train_dataset = TensorDataset(train_obs_v, train_act_v)
        # crea el dataloader                      
        train_dataloader = DataLoader(dataset = train_dataset, batch_size = self.batch_size)
        # entrega los datos
        return train_dataloader, reward_bound, reward_mean
    

### Prueba del agente (CartPole)

In [11]:
# selecciona un ambiente CartPole
env = gym.make("CartPole-v1")

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes para el iterador de lotes
BATCH_SIZE = 20
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# define el percentil para los episodios élite
PERCENTILE = 70

# Instancia un agente
agent = Agent(batch, PERCENTILE)

# entrega un conjunto de datos de los episodios élite de un  lote
dato = agent.take_action()
dato

(<torch.utils.data.dataloader.DataLoader at 0x7f39bc1daf10>,
 27.499999999999993,
 23.6)

### Entrenamiento CartPole

In [44]:
from torch.utils.tensorboard import SummaryWriter
# indica donde se escribirá el log de tensorboard
!tensorboard --logdir './runs'

# selecciona un ambiente CartPole
env = gym.make("CartPole-v1")

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes para el iterador de lotes
BATCH_SIZE = 20
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# define el percentil para los episodios élite
PERCENTILE = 70

# Instancia un agente
agent = Agent(batch, PERCENTILE)

#instancia writer para tensorboard
writer = SummaryWriter(comment="Entrenamiento de CartPole")
# agerga una grafo del modelo a tensorboard

#episode = next(batch)
#obs = episode.observation
#writer.add_graph(net, obs)

trainer = Trainer(model=net, writer=writer)

# ciclo de entrenamiento
min_reward = 200 # Para CartPole
max_iterations = 300
done = False
 
iter_no = 0

while not done:
    iter_no += 1
    # pide datos al agente
    dataloader, reward_bound, reward_mean = agent.take_action()
    # hace un paso de entrenamiento de la red
      
    trainer.fit(iter_no, dataloader, reward_bound, reward_mean)
    #trainer.save_checkpoint()
    #validation = trainer.validate(dataloaders=dataloader)
    if reward_mean > min_reward:
            print("Resuelto!")
            done = True
    if iter_no == max_iterations:
            print("Terminado por máximo número de iteraciones. No resuelto")
            done = True


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
1: pérdida promedio =0.685, recompensa promedio=28.1, cota recompensa=27.5
2: pérdida promedio =0.667, recompensa promedio=35.6, cota recompensa=41.8
3: pérdida promedio =0.649, recompensa promedio=32.4, cota recompensa=34.6
4: pérdida promedio =0.644, recompensa promedio=35.7, cota recompensa=40.9
5: pérdida promedio =0.626, recompensa promedio=38.1, cota recompensa=39.6
6: pérdida promedio =0.623, recompensa promedio=48.2, cota recompensa=54.5
7: pérdida promedio =0.595, recompensa promedio=80.2, cota recompensa=88.3
8: pérdida promedio =0.570, recompensa promedio=88

### Entrenamiento FrozenLake

In [13]:

# selecciona un ambiente CartPole
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes para el iterador de lotes
BATCH_SIZE = 100
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# define el percentil para los episodios élite
PERCENTILE = 70

# Instancia un agente
agent = Agent(batch, PERCENTILE)

trainer = Trainer(net)

# ciclo de entrenamiento
min_reward = 0.8 # Para FrozenLake
max_iterations = 100
done = False
 
iter_no = 0

while not done:
    iter_no += 1
    # pide datos al agente
    dataloader, reward_bound, reward_mean = agent.take_action()
    # hace un paso de entrenamiento de la red
      
    trainer.fit(iter_no, dataloader, reward_bound, reward_mean)
    #trainer.save_checkpoint()
    #validation = trainer.validate(dataloaders=dataloader)
    if reward_mean > min_reward:
            print("Resuelto!")
            done = True
    if iter_no == max_iterations:
            print("Terminado por máximo número de iteraciones. No resuelto")
            done = True


1: pérdida promedio =1.389, recompensa promedio=0.0, cota recompensa=0.0
2: pérdida promedio =1.386, recompensa promedio=0.0, cota recompensa=0.0
3: pérdida promedio =1.382, recompensa promedio=0.0, cota recompensa=0.0
4: pérdida promedio =1.382, recompensa promedio=0.0, cota recompensa=0.0
5: pérdida promedio =1.382, recompensa promedio=0.0, cota recompensa=0.0
6: pérdida promedio =1.379, recompensa promedio=0.0, cota recompensa=0.0
7: pérdida promedio =1.364, recompensa promedio=0.0, cota recompensa=0.0
8: pérdida promedio =1.356, recompensa promedio=0.0, cota recompensa=0.0
9: pérdida promedio =1.367, recompensa promedio=0.0, cota recompensa=0.0
10: pérdida promedio =1.368, recompensa promedio=0.0, cota recompensa=0.0
11: pérdida promedio =1.352, recompensa promedio=0.0, cota recompensa=0.0
12: pérdida promedio =1.347, recompensa promedio=0.0, cota recompensa=0.0
13: pérdida promedio =1.351, recompensa promedio=0.0, cota recompensa=0.0
14: pérdida promedio =1.349, recompensa promedi

## <span style="color:blue">Modificación para el  ambiente FrozenLake</span>

Más adelante en este curso volveremos a este ambiente para resolver las limitaciones del MEC con otros métodos de aprendizaje reforzado.

De momento haremos unas mejoras que ayuden al MEC a estimar la distribución de las acciones en el espacio de las observaciones. Haremos los siguiente:

* Lotes de episodios más largos. Pasaremos a 100 episodios por lote.
* Aplicaremos  el factor de descuento $\gamma$ a la recompensa. Asi espisodios más largos tendrán una menor recompensa y viceversa.Esto incrementa la variabilidad de la distribución de la recompensa.
* Mantendremos episodios élite por más largo tiempo.
* Decreceremos la rata de aprendizaje. Esto implica que la red neuronal tendrá más tiempo para ver en promedio mas muestras de entrenamiento.
* Mucho mayor tiempo de entrenamiento. 

Por favor revise, corra y modifique si lo considera necesario, el siguiente código.


## Clase Agent2

In [16]:
import numpy as np

from torch.utils.data import Dataset, TensorDataset, DataLoader

class Agent2(Agent):
    def __init__(self, batch_iterator, percentile, batch_size=1, gamma=0.9):
        super(Agent2, self).__init__(batch_iterator, percentile, batch_size)
        
        self.gamma = gamma
        self.full_batch = []
        
    def take_action(self): 
        # toma un lote de datos del iterador de lotes
        batch = next(self.batch_iterator)
        # agrega los datos que tenga preservados del episodio anterior
        batch = batch + self.full_batch 
        # filtro para modificar la recompensa. Episodios mas largos tiene mayor descuento 
        filter_fun = lambda s: s.reward * (self.gamma** len(s.steps))
        # extrae todas las recompensas del batch y hace una lista con ellas
        disc_rewards = list(map(filter_fun, batch))
        # calcula la cota inferior para extraer los episodios élite (por defecto percentil 70)
        reward_bound = np.percentile(disc_rewards, self.percentile)
        # calcula la recompensa promedio del lote de datos
        reward_mean = float(np.mean(disc_rewards))
        
        # extrae las observaciones y las respectivas acciones de los episodios élite
        train_obs = []
        train_act = []
        elite_batch = []
        
        for example, discounted_reward in zip(batch, disc_rewards):
            if discounted_reward > reward_bound:
                train_obs.extend(map(lambda step: step.observation,
                                     example.steps))
                train_act.extend(map(lambda step: step.action,
                                     example.steps))
                elite_batch.append(example)
        # guarda este batch élite para el siguiente episodio
        self.full_batch = elite_batch[-500:] # conserva los últimos 500 datos
            
        # convierte listas a tensores
        train_obs_v = torch.FloatTensor(train_obs)
        train_act_v = torch.LongTensor(train_act)
        # crea el dataset
        train_dataset = TensorDataset(train_obs_v, train_act_v)
        # crea el dataloader                      
        train_dataloader = DataLoader(dataset = train_dataset, batch_size = self.batch_size)
        # entrega los datos
        return train_dataloader, reward_bound, reward_mean
    

### Re-entrenamiento FrozenLake

In [17]:
from torch.utils.tensorboard import SummaryWriter
# indica donde se escribirá el log de tensorboard
tensorboard --logdir=runs

# selecciona un ambiente CartPole
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))

# extrae tamaños de acciones y observaciones en el ambiente
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
# define tamaño capa oculta de la red
HIDDEN_SIZE = 128
#instancia un objeto Net
net = Net(obs_size, HIDDEN_SIZE, n_actions)

# define tamaño de los lotes para el iterador de lotes
BATCH_SIZE = 100
# instancia un iterador Batch
batch = Batch(env, net, BATCH_SIZE)

# define el percentil para los episodios élite
PERCENTILE = 50

# Instancia un agente
agent = Agent2(batch, PERCENTILE)

# instancia writer
writer = SummaryWriter(comment="Entrenamiento de CartPole")

trainer = Trainer(model=net, writer=writer)

# ciclo de entrenamiento
min_reward = 0.8 # Para FrozenLake
max_iterations = 1000
done = False
 
iter_no = 0

while not done:
    iter_no += 1
    # pide datos al agente
    dataloader, reward_bound, reward_mean = agent.take_action()
    # hace un paso de entrenamiento de la red
      
    trainer.fit(iter_no, dataloader, reward_bound, reward_mean)
    #trainer.save_checkpoint()
    #validation = trainer.validate(dataloaders=dataloader)
    if reward_mean > min_reward:
            print("Resuelto!")
            done = True
    if iter_no == max_iterations:
            print("Terminado por máximo número de iteraciones. No resuelto")
            done = True

# envia al write cualquier cálculo pendiente
writer.flush()
# cierra el writer
close(writer)

1: pérdida promedio =1.360, recompensa promedio=0.0, cota recompensa=0.0
2: pérdida promedio =1.340, recompensa promedio=0.0, cota recompensa=0.0
3: pérdida promedio =1.369, recompensa promedio=0.0, cota recompensa=0.0
4: pérdida promedio =1.344, recompensa promedio=0.0, cota recompensa=0.0
5: pérdida promedio =1.319, recompensa promedio=0.0, cota recompensa=0.0
6: pérdida promedio =1.310, recompensa promedio=0.0, cota recompensa=0.0
7: pérdida promedio =1.288, recompensa promedio=0.0, cota recompensa=0.0
8: pérdida promedio =1.277, recompensa promedio=0.0, cota recompensa=0.0
9: pérdida promedio =1.270, recompensa promedio=0.1, cota recompensa=0.0
10: pérdida promedio =1.264, recompensa promedio=0.1, cota recompensa=0.0
11: pérdida promedio =1.263, recompensa promedio=0.1, cota recompensa=0.0
12: pérdida promedio =1.258, recompensa promedio=0.1, cota recompensa=0.0
13: pérdida promedio =1.245, recompensa promedio=0.1, cota recompensa=0.0
14: pérdida promedio =1.234, recompensa promedi